In [278]:
import pandas as pd
print("pandas loaded")

pandas loaded


In [302]:
df = pd.read_excel("flipkart_sales_data.xlsx")
df.drop(columns=['Order ID', 'Order Date'], inplace=True)
df

,Product Category,Sales Amount,Quantity Sold,Region,Customer Rating,Payment Method,Discount (%)
0,Home Decor,16295.0,2.0,East,1.5,UPI,10.0
1,Beauty & Health,1360.0,5.0,West,3.1,Credit Card,15.0
2,Electronics,5890.0,1.0,Central,1.5,Net Banking,5.0
3,Home Decor,12464.0,7.0,South,2.1,UPI,5.0
4,Grocery,11784.0,1.0,South,1.2,COD,10.0
...,...,...,...,...,...,...,...
1001,NaN,4880.0,NaN,South,NaN,NaN,NaN
1002,NaN,5257.0,NaN,West,NaN,NaN,NaN
1003,NaN,19597.0,NaN,Central,NaN,NaN,NaN
1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [280]:
df.isna().sum()

,0
Product Category,6
Sales Amount,2
Quantity Sold,6
Region,2
Customer Rating,6
Payment Method,6
Discount (%),6


In [281]:
df.columns[df.isna().any()].tolist()

['Product Category',
 'Sales Amount',
 'Quantity Sold',
 'Region',
 'Customer Rating',
 'Payment Method',
 'Discount (%)']

In [282]:
cat_columns = df.select_dtypes(include='object').columns.tolist()
cat_columns

['Product Category', 'Region', 'Payment Method']

In [283]:
for col in cat_columns:
  mode = df[col].mode()[0]
  df[col] = df[col].fillna(mode)

In [284]:
df.isna().sum()

,0
Product Category,0
Sales Amount,2
Quantity Sold,6
Region,0
Customer Rating,6
Payment Method,0
Discount (%),6


In [285]:
# num_columns = df.select_dtypes(exclude='object').columns.tolist()
# num_columns
num_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_columns
df[num_columns].describe()

,Sales Amount,Quantity Sold,Customer Rating,Discount (%)
count,1004.000000,1000.000000,1000.000000,1000.00000
mean,10085.825697,4.924000,3.016100,14.45000
std,5574.530275,2.612479,1.164675,7.30074
min,509.000000,1.000000,1.000000,5.00000
25%,5383.500000,3.000000,2.000000,10.00000
50%,9835.000000,5.000000,3.000000,15.00000
75%,14657.750000,7.000000,4.000000,20.00000
max,19992.000000,9.000000,5.000000,25.00000


In [286]:
for col in num_columns:
  mean_value = df[col].mean()
  df[col] = df[col].fillna(mean_value)

In [287]:
df.isna().sum()

,0
Product Category,0
Sales Amount,0
Quantity Sold,0
Region,0
Customer Rating,0
Payment Method,0
Discount (%),0


In [288]:
outlier_info = {}

for col in num_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_info[col] = outliers.shape[0]

    print(f"{col}: {outliers.shape[0]} outliers")

Sales Amount: 0 outliers
Quantity Sold: 0 outliers
Customer Rating: 0 outliers
Discount (%): 0 outliers


In [289]:
df[(df['Sales Amount'] < lower_bound) | (df['Sales Amount'] > upper_bound)]
#df[(df['cat_col_1'] < lower_bound) | (df['cat_col_1'] > upper_bound)] in general form

,Product Category,Sales Amount,Quantity Sold,Region,Customer Rating,Payment Method,Discount (%)
0,Home Decor,16295.000000,2.000,East,1.5000,UPI,10.00
1,Beauty & Health,1360.000000,5.000,West,3.1000,Credit Card,15.00
2,Electronics,5890.000000,1.000,Central,1.5000,Net Banking,5.00
3,Home Decor,12464.000000,7.000,South,2.1000,UPI,5.00
4,Grocery,11784.000000,1.000,South,1.2000,COD,10.00
...,...,...,...,...,...,...,...
1001,Grocery,4880.000000,4.924,South,3.0161,COD,14.45
1002,Grocery,5257.000000,4.924,West,3.0161,COD,14.45
1003,Grocery,19597.000000,4.924,Central,3.0161,COD,14.45
1004,Grocery,10085.825697,4.924,South,3.0161,COD,14.45


In [290]:
df.dtypes

,0
Product Category,object
Sales Amount,float64
Quantity Sold,float64
Region,object
Customer Rating,float64
Payment Method,object
Discount (%),float64


In [291]:
# df['Order Date'] = pd.to_datetime(df['Order Date'])

In [292]:
df.duplicated().sum()

np.int64(1)

In [293]:
df.drop_duplicates(inplace=True)

In [294]:
df.duplicated().sum()

np.int64(0)

In [301]:
#encodin - one hot encoding
df_one__hot_encoding = pd.get_dummies(df, columns=cat_columns)
df_one__hot_encoding = df_one__hot_encoding.astype(int)
df_one__hot_encoding

,Sales Amount,Quantity Sold,Customer Rating,Discount (%),Product Category_Beauty & Health,Product Category_Books,Product Category_Electronics,Product Category_Fashion,Product Category_Grocery,Product Category_Home Decor,...,Region_Central,Region_East,Region_North,Region_South,Region_West,Payment Method_COD,Payment Method_Credit Card,Payment Method_Debit Card,Payment Method_Net Banking,Payment Method_UPI
0,16295,2,1,10,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,1360,5,3,15,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,5890,1,1,5,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,12464,7,2,5,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,11784,1,1,10,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,13972,4,3,14,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
1001,4880,4,3,14,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
1002,5257,4,3,14,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
1003,19597,4,3,14,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [299]:
#feature scaling
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_one__hot_encoding[num_columns] = scaler.fit_transform(df_one__hot_encoding[num_columns])
df_one__hot_encoding.head()

,Sales Amount,Quantity Sold,Customer Rating,Discount (%),Product Category_Beauty & Health,Product Category_Books,Product Category_Electronics,Product Category_Fashion,Product Category_Grocery,Product Category_Home Decor,...,Region_Central,Region_East,Region_North,Region_South,Region_West,Payment Method_COD,Payment Method_Credit Card,Payment Method_Debit Card,Payment Method_Net Banking,Payment Method_UPI
0,0.810245,0.125,0.00,0.25,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,0.043679,0.500,0.50,0.50,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0.276189,0.000,0.00,0.00,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,0.613612,0.750,0.25,0.00,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,0.578710,0.000,0.00,0.25,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
